In [1]:
# copyright (c) 2021 PaddlePaddle Authors. All Rights Reserve.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# coding=utf-8

import numpy as np
import json
# import matplotlib
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

def load_data():
    # 从文件导入数据
    datafile = './housing.data'
    data = np.fromfile(datafile, sep=' ')

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                          'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算训练集的最大值，最小值，平均值
    maximums, minimums, avgs = training_data.max(axis=0), training_data.min(axis=0), training_data.sum(axis=0) / training_data.shape[0]

    # 对数据进行归一化处理
    for i in range(feature_num):
        #print(maximums[i], minimums[i], avgs[i])
        data[:, i] = (data[:, i] - minimums[i]) / (maximums[i] - minimums[i])

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data

class Network(object):
    def __init__(self, num_of_weights):
        # 随机产生w的初始值
        # 为了保持程序每次运行结果的一致性，此处设置固定的随机数种子
        #np.random.seed(0)
        self.w = np.random.randn(num_of_weights, 1)
        self.b = 0.
        
    def forward(self, x):
        z = np.dot(x, self.w) + self.b
        return z
    
    def loss(self, z, y):
        error = z - y
        num_samples = error.shape[0]
        cost = error * error
        cost = np.sum(cost) / num_samples
        return cost
    
    def gradient(self, x, y):
        z = self.forward(x)
        N = x.shape[0]
        gradient_w = 1. / N * np.sum((z-y) * x, axis=0)
        gradient_w = gradient_w[:, np.newaxis]
        gradient_b = 1. / N * np.sum(z-y)
        return gradient_w, gradient_b
    
    def update(self, gradient_w, gradient_b, eta = 0.01):
        self.w = self.w - eta * gradient_w
        self.b = self.b - eta * gradient_b
            
                
    def train(self, training_data, num_epochs, batch_size=10, eta=0.01):
        n = len(training_data)
        losses = []
        for epoch_id in range(num_epochs):
            # 在每轮迭代开始之前，将训练数据的顺序随机打乱
            # 然后再按每次取batch_size条数据的方式取出
            np.random.shuffle(training_data)
            # 将训练数据进行拆分，每个mini_batch包含batch_size条的数据
            mini_batches = [training_data[k:k+batch_size] for k in range(0, n, batch_size)]
            for iter_id, mini_batch in enumerate(mini_batches):
                #print(self.w.shape)
                #print(self.b)
                x = mini_batch[:, :-1]
                y = mini_batch[:, -1:]
                a = self.forward(x)
                loss = self.loss(a, y)
                gradient_w, gradient_b = self.gradient(x, y)
                self.update(gradient_w, gradient_b, eta)
                losses.append(loss)
                print('Epoch {:3d} / iter {:3d}, loss = {:.4f}'.
                                 format(epoch_id, iter_id, loss))
        
        return losses
def train():
    # 获取数据
    train_data, test_data = load_data()

    # 创建网络
    net = Network(13)
    # 启动训练
    losses = net.train(train_data, num_epochs=50, batch_size=100, eta=0.1)

    # 画出损失函数的变化趋势
    # plot_x = np.arange(len(losses))
    # plot_y = np.array(losses)
    # plt.plot(plot_x, plot_y)
    # plt.show()

def plot_3D_neural_work_weight():
    # 获取数据
    training_data, test_data = load_data()
    x = training_data[:, :-1]
    y = training_data[:, -1:]

    net = Network(13)
    # losses = []
    # #只画出参数w5和w9在区间[-160, 160]的曲线部分，以及包含损失函数的极值
    # w5 = np.arange(-160.0, 160.0, 1.0)
    # w9 = np.arange(-160.0, 160.0, 1.0)
    # losses = np.zeros([len(w5), len(w9)])

    # #计算设定区域内每个参数取值所对应的Loss
    # for i in range(len(w5)):
    #     for j in range(len(w9)):
    #         net.w[5] = w5[i]
    #         net.w[9] = w9[j]
    #         z = net.forward(x)
    #         loss = net.loss(z, y)
    #         losses[i, j] = loss

    # #使用matplotlib将两个变量和对应的Loss作3D图
    # fig = plt.figure()
    # ax = Axes3D(fig)

    # w5, w9 = np.meshgrid(w5, w9)

    # ax.plot_surface(w5, w9, losses, rstride=1, cstride=1, cmap='rainbow')
    # plt.show()

if __name__ == '__main__':
    # plot_3D_neural_work_weight()
    train()

Epoch   0 / iter   0, loss = 1.7083
Epoch   0 / iter   1, loss = 0.9106
Epoch   0 / iter   2, loss = 0.3617
Epoch   0 / iter   3, loss = 0.2644
Epoch   0 / iter   4, loss = 0.1828
Epoch   1 / iter   0, loss = 0.2243
Epoch   1 / iter   1, loss = 0.1619
Epoch   1 / iter   2, loss = 0.1134
Epoch   1 / iter   3, loss = 0.1812
Epoch   1 / iter   4, loss = 0.1741
Epoch   2 / iter   0, loss = 0.1762
Epoch   2 / iter   1, loss = 0.1234
Epoch   2 / iter   2, loss = 0.1217
Epoch   2 / iter   3, loss = 0.1396
Epoch   2 / iter   4, loss = 0.0783
Epoch   3 / iter   0, loss = 0.1509
Epoch   3 / iter   1, loss = 0.1291
Epoch   3 / iter   2, loss = 0.1250
Epoch   3 / iter   3, loss = 0.1201
Epoch   3 / iter   4, loss = 0.2345
Epoch   4 / iter   0, loss = 0.1392
Epoch   4 / iter   1, loss = 0.1254
Epoch   4 / iter   2, loss = 0.0953
Epoch   4 / iter   3, loss = 0.1571
Epoch   4 / iter   4, loss = 0.1610
Epoch   5 / iter   0, loss = 0.1081
Epoch   5 / iter   1, loss = 0.1352
Epoch   5 / iter   2, loss =